In [17]:
## #----imports
import math
from math import sqrt, acos, degrees
import numpy as np
# from sympy import symbols, Eq, solve
import csv
import os
from datetime import datetime

In [ ]:
# Define known variables
x1, y1, z1 = 1.0, 15.0, 8.0  # Example coordinates of the first point
d1 = 1.0  # Distance from the first point to the second
phi1 = 22.0  # Angle in degrees clockwise
xb, yb, zb = 5.0, 8.0, 9.0  # Coordinates of the base point
d3 = 8.0  # Example value for distance d3
d4 = 1.0  # Example value for distance d4

 <!-- a code that calculate distance, spheare, circuil and Trim points. Using python in a few steps:
1. I have a point (x_1,y_1,z_3), I have a length d_1 and Angle phi_1. I want to find the point (x_2,y_2,z_2) that is in a distance d_1 and angle phi_1 clock wise  from (x_1,y_1,z_1). 
2. I have (x_2,y_2,z_2) and (x_b,y_b,z_b) I want to calculate d_2 the distance between both of them.
3. I have d_2, d_3, d_4 and I want to calculate the andle between  d_3, d_4 using cos-sin smathematicle sentense.

(x_1,y_1,z_3), (x_b,y_b,z_b), phi_1, d_3, d_4 are known variblse  -->

In [19]:
def calculate_second_point(x1, y1, z1, d1, phi1):
    # Convert phi1 to radians for calculations
    phi1_rad = math.radians(phi1)
    x2 = x1 + d1 * math.cos(phi1_rad)
    y2 = y1 + d1 * math.sin(phi1_rad)
    z2 = z1  # Assuming z2 remains the same since no vertical angle is provided
    return x2, y2, z2

def calculate_distance(x1, y1, z1, x2, y2, z2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2 + (z2 - z1) ** 2)

def calculate_angle_between_vectors(d3, d4, d2):
    # Using the cosine rule
    try:
        angle_rad = math.acos((d3 ** 2 + d4 ** 2 - d2 ** 2) / (2 * d3 * d4))
        angle_deg = math.degrees(angle_rad)
        return angle_rad, angle_deg
    except ValueError:
        return None, "Invalid input values for calculating angle. Check distances."



# Step 1: Calculate the second point
x2, y2, z2 = calculate_second_point(x1, y1, z1, d1, phi1)
print(f"Second point: ({x2:.2f}, {y2:.2f}, {z2:.2f})")

# Step 2: Calculate the distance between the second point and the base point
d2 = calculate_distance(x2, y2, z2, xb, yb, zb)
print(f"Distance between second point and base point: {d2:.2f}")

# Step 3: Calculate the angle between d3 and d4
angle_rad, angle_deg = calculate_angle_between_vectors(d3, d4, d2)
if angle_rad is not None:
    print(f"Angle between d3 and d4: {angle_deg:.2f} degrees")
else:
    print(angle_deg)

phi_2 = angle_deg

Second point: (1.93, 15.37, 8.00)
Distance between second point and base point: 8.05
Angle between d3 and d4: 89.38 degrees


<!-- lets call the angle from before phi_2.
1. I want to have a circuil equestion by using (x_2,y_2,z_2) as it's center and d_1  is the radius (the distance between(x_2,y_2,z_2) and (x_1,y_1,z_1)).

2.  I want to have a sphear that its center is (x_b,y_b,z_b) and it's radius is d_2 (the distance between (x_b,y_b,z_b) and (x_1,y_1,z_1) surface sphear)

3. I want to calculate all the Trim points between teh sphear and the circuil. -->

In [20]:
# #-----versition#3
def calculate_second_point(x1, y1, z1, d1, phi1):
    phi1_rad = math.radians(phi1)
    x2 = x1 + d1 * math.cos(phi1_rad)
    y2 = y1 + d1 * math.sin(phi1_rad)
    return x2, y2, z1

def calculate_distance(x1, y1, z1, x2, y2, z2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2 + (z2 - z1) ** 2)

def calculate_trim_points(circle_center, circle_radius, sphere_center, sphere_radius):
    cx, cy, cz = circle_center
    sx, sy, sz = sphere_center

    # Distance from sphere center to plane of circle
    dz = cz - sz
    if abs(dz) > sphere_radius:
        return []  # No intersection

    # Radius of sphere's cross-section at that plane
    r_cross = math.sqrt(sphere_radius**2 - dz**2)

    # Now we have two circles in XY plane:
    # 1. center=(cx,cy), radius=circle_radius
    # 2. center=(sx,sy), radius=r_cross
    dx = sx - cx
    dy = sy - cy
    d = math.sqrt(dx**2 + dy**2)

    # No intersection cases
    if d > circle_radius + r_cross or d < abs(circle_radius - r_cross):
        return []

    # Circle-circle intersection in 2D
    a = (circle_radius**2 - r_cross**2 + d**2) / (2*d)
    h = math.sqrt(max(circle_radius**2 - a**2, 0))
    xm = cx + a * dx / d
    ym = cy + a * dy / d

    rx = -dy * (h / d)
    ry = dx * (h / d)

    p1 = (xm + rx, ym + ry, cz)
    p2 = (xm - rx, ym - ry, cz)
    return [p1, p2]

# Calculate second point
x2, y2, z2 = calculate_second_point(x1, y1, z1, d1, phi1)

# Calculate d2
d2 = calculate_distance(x2, y2, z2, xb, yb, zb)

# # Setup centers for calculations
# circle_center = np.array([x2, y2, z2])
# sphere_center = np.array([xb, yb, zb])

# Compute intersections
trim_points = calculate_trim_points((x2, y2, z2), d1, (xb, yb, zb), d2)

print("Trim points between circle and sphere:")
for pt in trim_points:
    print(f"({pt[0]:.2f}, {pt[1]:.2f}, {pt[2]:.2f})")

Trim points between circle and sphere:
(2.87, 15.70, 8.00)
(1.03, 14.93, 8.00)


In [21]:
from math import acos, degrees, sqrt

# Function to calculate distance between two points
def calculate_distance(point1, point2):
    return sqrt((point1[0] - point2[0])**2 + 
                (point1[1] - point2[1])**2 + 
                (point1[2] - point2[2])**2)

# Define distances and coordinates lists
x4, y4, z4 = [], [], []
d4_list, d5_list = [], []

# Distance between (x1, y1, z1) and (x2, y2, z2)
d1 = calculate_distance((x1, y1, z1), (x2, y2, z2))

print("Angle calculations for each trim point:")
for point in trim_points:
    try:
        # Trim point coordinates (already floats)
        x, y, z = point
        x4.append(x)
        y4.append(y)
        z4.append(z)
        
        # Distances from trim point
        d5_val = calculate_distance((x1, y1, z1), (x, y, z))  # Distance to (x1, y1, z1)
        d4_val = calculate_distance((x2, y2, z2), (x, y, z))  # Distance to (x2, y2, z2)
        d5_list.append(d5_val)
        d4_list.append(d4_val)
        
        # Calculate angle phi_3 using the Law of Cosines
        if d1 > 0 and d4_val > 0:  
            cos_phi3 = (d1**2 + d4_val**2 - d5_val**2) / (2 * d1 * d4_val)
            cos_phi3 = max(-1, min(1, cos_phi3))  # Clamp to [-1, 1]
            phi_3 = degrees(acos(cos_phi3))
            print(f"Trim point: ({x:.2f}, {y:.2f}, {z:.2f}), Phi_3: {phi_3:.2f}°")
        else:
            print(f"Trim point: ({x:.2f}, {y:.2f}, {z:.2f}), Phi_3: Undefined (d1 or d4 is zero)")
    except Exception as e:
        print(f"Error processing trim point {point}: {e}")


Angle calculations for each trim point:
Trim point: (2.87, 15.70, 8.00), Phi_3: 177.03°
Trim point: (1.03, 14.93, 8.00), Phi_3: 4.21°


<!-- Lets call trim_points as (x_4_i,y_4_i) while i passing all the Trimp Points we found.

Let's loop for value in trim_points:

1. I want to calculate the disstance between (x_1,y_1) and the point (x_4_i,y_4_i) lets call it d5

2. the disstance between (x_2,y_2) and the point (x_4_i,y_4_i) is d4.

3. The distance between (x_1,y_1) and the point (x_2,y_2) is d1. 

4. I want to calculate the angle from d1 to d4 clock wise (very inportent) using 
Law of cosines. Lets call this angle phi_3 -->


<!-- Using the trim_point (x_4,y_4,z_4)[0]  and phi_3[0]
We calculate the distance :
1. between (x2,y2) and (xb,yb) let's call it d6
2. between (xb,yb) and (x4,y4) lets call it d7
3. between (x4,y4) and (x2,y2)  lets call it d8

use the cosine equestion to calculate the angle between f7 and d8 -->

In [22]:
from math import sqrt, acos, degrees

# Function to calculate 2D distance
def calculate_distance_2d(point1, point2):
    return sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

# Function to calculate the angle using the cosine law
def calculate_angle_cosine(d1, d2, d3):
    if d1 * d2 == 0:
        raise ValueError("Invalid distances, cannot calculate angle.")
    cos_theta = (d1**2 + d2**2 - d3**2) / (2 * d1 * d2)
    cos_theta = max(min(cos_theta, 1), -1)  # Clamp to [-1, 1]
    return degrees(acos(cos_theta))

# Coordinates of first trim point (already floats)
x4, y4, z4 = trim_points[0]

# Calculate distances
d6 = calculate_distance_2d((x2, y2), (xb, yb))  # (x2,y2) ↔ (xb,yb)
d7 = calculate_distance_2d((xb, yb), (x4, y4))  # (xb,yb) ↔ (x4,y4)
d8 = calculate_distance_2d((x4, y4), (x2, y2))  # (x4,y4) ↔ (x2,y2)

# Calculate angle
try:
    angle_f7_d8 = calculate_angle_cosine(d7, d8, d6)
    print(f"Distances: d6: {d6:.2f}, d7: {d7:.2f}, d8: {d8:.2f}")
    print(f"Angle between d7 and d8: {angle_f7_d8:.2f}°")
except ValueError as e:
    print(f"Error calculating angle: {e}")

phi_4 = angle_f7_d8


Distances: d6: 7.99, d7: 7.99, d8: 1.00
Angle between d7 and d8: 86.41°


<!-- Calculate the angle between d7 and d6. call it phi_5 -->

In [23]:
# Calculate angles
try:
    phi_5 = calculate_angle_cosine(d7, d6, d8)  # Angle between d7 and d6
    print(f"Distances: d6: {d6:.2f}, d7: {d7:.2f}, d8: {d8:.2f}")
    print(f"Angle between d7 and d6 (phi_5): {phi_5:.2f}°")
except ValueError as e:
    print(f"Error calculating angle: {e}")


Distances: d6: 7.99, d7: 7.99, d8: 1.00
Angle between d7 and d6 (phi_5): 7.18°


<!-- Find the point that is distance from (x2,y2) in d6 and it have yb as a known value for y value. This point will be (x5,y5). chack if one of the  values is xb and if so ignore it  -->

In [24]:
from math import sqrt

def find_x_given_y_exclude_xb(x2, y2, yb, d6, xb):
    try:
        term = d6**2 - (yb - y2)**2
        if term < 0:
            raise ValueError("No real solution exists for the given inputs.")
        
        sqrt_term = sqrt(term)
        x5_positive = x2 + sqrt_term
        x5_negative = x2 - sqrt_term
        
        # Exclude xb from the results
        solutions = []
        if abs(x5_positive - xb) > 1e-6:  # Tolerance for floating-point comparisons
            solutions.append(x5_positive)
        if abs(x5_negative - xb) > 1e-6:
            solutions.append(x5_negative)
        
        return solutions
    except Exception as e:
        print(f"Error: {e}")
        return []

x5_solutions = find_x_given_y_exclude_xb(x2, y2, yb, d6, xb)
if x5_solutions:
    print(f"Possible x-coordinates for the point (excluding xb={xb}): {x5_solutions}")
    for x5 in x5_solutions:
        print(f"Point: ({x5:.2f}, {yb:.2f})")
else:
    print("No solution found.")


Possible x-coordinates for the point (excluding xb=5.0): [-1.1456322908664256]
Point: (-1.15, 8.00)


<!-- The distance between (x5,y5) to (xb,yb) call it d7 
use the cosine sentense to calculate the angle between d6 and d7 (the values are d6,d6 and d7)
This angle will call phi_6 -->

In [25]:
from math import acos, degrees, sqrt

# Function to calculate distance between two points
def calculate_distance(point1, point2):
    return sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

# Function to calculate phi_6
def calculate_phi_6(x2, y2, x5, y5, xb, yb):
    try:
        # Calculate distances
        d6 = calculate_distance((x2, y2), (x5, y5))
        d7 = calculate_distance((x5, y5), (xb, yb))
        
        # Use the Law of Cosines to calculate the angle
        if d6 > 0:  # Ensure d6 is non-zero to avoid division by zero
            cos_phi_6 = (d6**2 + d6**2 - d7**2) / (2 * d6 * d6)
            # Clamp cos_phi_6 to the range [-1, 1] to handle floating-point inaccuracies
            cos_phi_6 = max(-1, min(1, cos_phi_6))
            phi_6 = degrees(acos(cos_phi_6))
            return d6, d7, phi_6
        else:
            raise ValueError("d6 is zero, cannot calculate angle.")
    except Exception as e:
        print(f"Error: {e}")
        return None, None, None

# Example usage
x2, y2 = 5, 5  # Coordinates of point 2
x5, y5 = 8, 7  # Coordinates of point 5
xb, yb = 6, 7  # Coordinates of point b

d6, d7, phi_6 = calculate_phi_6(x2, y2, x5, y5, xb, yb)
if phi_6 is not None:
    print(f"Distance d6: {d6:.2f}")
    print(f"Distance d7: {d7:.2f}")
    print(f"Angle phi_6: {phi_6:.2f}°")
else:
    print("Failed to calculate phi_6.")


Distance d6: 3.61
Distance d7: 2.00
Angle phi_6: 32.20°


<!-- Sum phi_5 + phi_6 - 60 dgrees -->

In [26]:
phi_7 = phi_5 + phi_6 - 60
phi_7

-20.619411759765207

<!-- Let's calculate phi_8 angle by using the projection the trapezoid between (𝑥1,𝑦1,𝑧1), (𝑥4,𝑦4,𝑧4), (xb,yb,zb) and the base
 -->


In [27]:
phi_8 = 360 - (phi_2 + 90 + 90)
print(f"phi_8: {phi_8:.2f}°")

phi_8: 90.62°


<!-- y9=yb and z9 = z4
To calculate the angle between arm (y2,z2),(yb,zb) to arm (yb,zb),(y9,z9) 
call it phi_9. -->

In [28]:
from math import acos, degrees, sqrt

# Function to calculate distance in 2D
def calculate_distance_2d(coord1, coord2):
    return sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

# Function to calculate angle using cosine law
def calculate_angle_cosine(d1, d2, d3):
    if d1 * d2 == 0:
        raise ValueError("Invalid distances, cannot calculate angle.")
    cos_theta = (d1**2 + d2**2 - d3**2) / (2 * d1 * d2)
    cos_theta = max(min(cos_theta, 1), -1)  # Clamp to avoid numerical issues
    return degrees(acos(cos_theta))

# Known coordinates (z9 is the z-coordinate of the first trim point)
y9, z9 = yb, trim_points[0][2]

# Calculate distances
d_yb_z2 = calculate_distance_2d((y2, z2), (yb, zb))  # Distance (y2,z2) ↔ (yb,zb)
d_yb_z9 = calculate_distance_2d((yb, zb), (y9, z9))  # Distance (yb,zb) ↔ (y9,z9)
d_z2_z9 = calculate_distance_2d((y2, z2), (y9, z9))  # Distance (y2,z2) ↔ (y9,z9)

# Calculate phi_9
try:
    phi_9 = calculate_angle_cosine(d_yb_z2, d_yb_z9, d_z2_z9)
    print(f"Distances: d_yb_z2: {d_yb_z2:.2f}, d_yb_z9: {d_yb_z9:.2f}, d_z2_z9: {d_z2_z9:.2f}")
    print(f"Angle between arms (phi_9): {phi_9:.2f}°")
except ValueError as e:
    print(f"Error calculating angle: {e}")


Distances: d_yb_z2: 2.24, d_yb_z9: 1.00, d_z2_z9: 2.00
Angle between arms (phi_9): 63.43°


In [30]:

# Your data variables (example placeholders)
L1, L2, L3, L4, L5, L6 = 1, 2, 3, 4, 5, 6
x1, y1, z1 = 1, 2, 3
x4, y4, z4 = 4, 5, 6
xa, ya, za = 7, 8, 9
xb, yb, zb = 10, 11, 12
phi_1, phi_2, phi_3, phi_4, phi_5, phi_6, phi_7, phi_8, phi_9 = 10, 20, 30, 40, 50, 60, 70, 80, 90
y2, z2 = 13, 14
y9, z9 = 15, 16
d_yb_z2, d_yb_z9, d_z2_z9 = 17, 18, 19

# Get current timestamp
current_time = datetime.now()

# Prepare header and data row
headers = [
    'timestamp',
    'L1', 'L2', 'L3', 'L4', 'L5', 'L6',
    'x1', 'y1', 'z1',
    'x4', 'y4', 'z4',
    'xa', 'ya', 'za',
    'xb', 'yb', 'zb',
    'phi_1', 'phi_2', 'phi_3', 'phi_4', 'phi_5', 'phi_6', 'phi_7', 'phi_8', 'phi_9',
    'y2', 'z2',
    'y9', 'z9',
    'd_yb_z2', 'd_yb_z9', 'd_z2_z9'
]

data_row = [
    current_time.isoformat(),
    L1, L2, L3, L4, L5, L6,
    x1, y1, z1,
    x4, y4, z4,
    xa, ya, za,
    xb, yb, zb,
    phi_1, phi_2, phi_3, phi_4, phi_5, phi_6, phi_7, phi_8, phi_9,
    y2, z2,
    y9, z9,
    d_yb_z2, d_yb_z9, d_z2_z9
]

file_path = 'geo_data.csv'

# Check if file exists to decide if we write headers
write_header = not os.path.exists(file_path)

with open(file_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if write_header:
        writer.writerow(headers)
    writer.writerow(data_row)

print(f"Data has been saved to {file_path}")

Data has been saved to geo_data.csv
